# Skyrme example

In [1]:
import o2sclpy

Link the O$_2$scl library

In [2]:
link=o2sclpy.linker()
link.link_o2scl_o2graph(True,True)

Get the value of $\hbar c$ from an O$_2$scl find_constants object

In [3]:
fc=o2sclpy.find_constants(link)
hc=fc.find_unique('hbarc','MeV*fm')
print('hbarc = %7.6e' % (hc))

hbarc = 1.973270e+02


Get a copy (a pointer to) the O$_2$scl unit conversion object

In [4]:
cu=link.o2scl_settings.get_convert_units()

Create neutron and proton objects and set their spin degeneracy and
masses. The masses are expected to be in units of inverse
femtometers.

In [5]:
neut=o2sclpy.fermion(link)
neut.g=2.0
neut.m=cu.convert('g','1/fm',fc.find_unique('massneutron','g'))

In [6]:
prot=o2sclpy.fermion(link)
prot.g=2.0
prot.m=cu.convert('g','1/fm',fc.find_unique('massproton','g'))

Create the Skyrme EOS object and load the NRAPR parameterization

In [7]:
sk=o2sclpy.eos_had_skyrme(link)
o2sclpy.skyrme_load(link,sk,'NRAPR',False,0)

Compute nuclear saturation and output the saturation density
and binding energy

In [8]:
sk.saturation()
print('NRAPR: n0=%7.6e 1/fm^3, E/A=%7.6e MeV' % (sk.n0,sk.eoa*hc))
print('')

NRAPR: n0=1.605785e-01 1/fm^3, E/A=-1.585619e+01 MeV



Create the nstar_cold object for automatically computing the
beta-equilibrium EOS and solving the TOV equations.

In [9]:
nc=o2sclpy.nstar_cold(link)

Let the nstar_cold object know we want to use the NRAPR EOS

In [10]:
nc.set_eos(sk)

Compute the EOS

In [11]:
ret1=nc.calc_eos(0.01)

Summarize the columns in the EOS table

In [12]:
eos_table=nc.get_eos_results()
print('EOS table:')
for i in range(0,eos_table.get_ncolumns()):
    col=eos_table.get_column_name(i)
    unit=eos_table.get_unit(col)
    print('Column',i,str(col,'UTF-8'),str(unit,'UTF-8'))
print('')

EOS table:
Column 0 ed 1/fm^4
Column 1 pr 1/fm^4
Column 2 nb 1/fm^3
Column 3 mun 1/fm
Column 4 mup 1/fm
Column 5 mue 1/fm
Column 6 nn 1/fm^3
Column 7 np 1/fm^3
Column 8 ne 1/fm^3
Column 9 kfn 1/fm
Column 10 kfp 1/fm
Column 11 kfe 1/fm
Column 12 fcs2 
Column 13 dednb_Ye 1/fm
Column 14 dPdnb_Ye 1/fm
Column 15 cs2 
Column 16 logp 
Column 17 loge 
Column 18 s 1/fm
Column 19 urca 1/fm^4
Column 20 ad_index 



In [19]:
# from https://stackoverflow.com/questions/24277488/in-python-how-to-capture-
# the-stdout-from-a-c-shared-library-to-a-variable
import ctypes
import os
import sys
import threading

# Create pipe and dup2() the write end of it on top of stdout, saving a copy
# of the old stdout
#stdout_fileno = sys.stdout.fileno()
stdout_fileno=1

stdout_save = os.dup(stdout_fileno)
stdout_pipe = os.pipe()
os.dup2(stdout_pipe[1], stdout_fileno)
os.close(stdout_pipe[1])

captured_stdout = ''
def drain_pipe():
    global captured_stdout
    while True:
        data = os.read(stdout_pipe[0], 1024)
        if not data:
            break
        captured_stdout += str(data,'UTF-8')

t = threading.Thread(target=drain_pipe)
t.start()

ret2=nc.calc_nstar()

# Close the write end of the pipe to unblock the reader thread and trigger it
# to exit
os.close(stdout_fileno)
t.join()

# Clean up the pipe and restore the original stdout
os.close(stdout_pipe[0])
os.dup2(stdout_save, stdout_fileno)
os.close(stdout_save)
print('y')
print(captured_stdout)

y
Mass versus radius mode.
Central P: 7.0000e-07 (Msun/km^3), M: 1.1194e-01 (Msun), R: 4.9240e+01 (km)
Central P: 7.7000e-07 (Msun/km^3), M: 1.1726e-01 (Msun), R: 4.1158e+01 (km)
Central P: 8.4700e-07 (Msun/km^3), M: 1.2311e-01 (Msun), R: 3.5499e+01 (km)
Central P: 9.3170e-07 (Msun/km^3), M: 1.2954e-01 (Msun), R: 3.1326e+01 (km)
Central P: 1.0249e-06 (Msun/km^3), M: 1.3650e-01 (Msun), R: 2.8161e+01 (km)
Central P: 1.1274e-06 (Msun/km^3), M: 1.4405e-01 (Msun), R: 2.5677e+01 (km)
Central P: 1.2401e-06 (Msun/km^3), M: 1.5217e-01 (Msun), R: 2.3694e+01 (km)
Central P: 1.3641e-06 (Msun/km^3), M: 1.6091e-01 (Msun), R: 2.2080e+01 (km)
Central P: 1.5005e-06 (Msun/km^3), M: 1.7029e-01 (Msun), R: 2.0747e+01 (km)
Central P: 1.6506e-06 (Msun/km^3), M: 1.8033e-01 (Msun), R: 1.9634e+01 (km)
Central P: 1.8156e-06 (Msun/km^3), M: 1.9107e-01 (Msun), R: 1.8693e+01 (km)
Central P: 1.9972e-06 (Msun/km^3), M: 2.0254e-01 (Msun), R: 1.7893e+01 (km)
Central P: 2.1969e-06 (Msun/km^3), M: 2.1479e-01 (Msun), R: 1

Compute the M-R curve using the TOV equations

In [14]:
# ret2=nc.calc_nstar()

Get the table for the TOV results

In [15]:
tov_table=nc.get_tov_results()
print('')

Summarize the columns in the TOV table

In [16]:
print('TOV table:')
for i in range(0,tov_table.get_ncolumns()):
    col=tov_table.get_column_name(i)
    unit=tov_table.get_unit(col)
    print('Column',i,str(col,'UTF-8'),str(unit,'UTF-8'))
print('')

TOV table:
Column 0 gm Msun
Column 1 r km
Column 2 bm Msun
Column 3 pr 1/fm^4
Column 4 ed 1/fm^4
Column 5 nb 1/fm^3
Column 6 sg 1/km
Column 7 rs 
Column 8 dmdr Msun/km
Column 9 dlogpdr 1/km
Column 10 dbmdr Msun/km

